In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from nfstream import NFStreamer, NFPlugin
import nfstream

In [2]:
count = 0
lowflow = 0
time = None
pkt_count = 0
freq_alarm = 0
lowaaf = 0

In [3]:
class PacketSizePerFlow(NFPlugin):

    def on_init(self, pkt, flow):
        global lowflow
        global time
        flow.expiration_id = 0
        flow.udps.packet_count = 1
        lowflow += 1
        flow.udps.is_subtracted = False
        flow.udps.alarm = False
        flow.udps.lowflow = lowflow
        if time is None:
            time = pkt.time

    def on_update(self, pkt, flow):
        global lowflow
        global time

        flow.udps.lowflow = lowflow
        flow.udps.packet_count = 1

        if flow.udps.packet_count > 4 and not flow.is_subtracted:
            lowflow -= 1
            flow.udps.is_subtracted = True



        if lowflow > 100:
            flow.udps.expiration_id = -1 # -1 value force expiration
            flow.udps.alarm = True
            lowflow = 0
            time = None

        if time is None:
            time = pkt.time
        time_diff = pkt.time - time

        if time_diff > 10000:
            lowflow = 0
            time = None

In [4]:
class AAF(NFPlugin):

    def on_init(self, pkt, flow):
        global lowaaf
        global time
        flow.expiration_id = 0
        flow.udps.aaf = 0
        if pkt.direction == 0:
                flow.udps.src2dst_size = pkt.payload_size
                flow.udps.dst2src_size = 0
        elif pkt.direction == 1:
                flow.udps.src2dst_size = 0
                flow.udps.dst2src_size = pkt.payload_size
        flow.udps.alarm = False
        flow.udps.lowaaf = lowaaf
        if time is None:
            time = pkt.time

            
    def on_update(self, pkt, flow):
        global lowaaf
        global time

        
        if pkt.direction == 0:
            flow.udps.src2dst_size += pkt.payload_size
        elif pkt.direction == 1:
            flow.udps.dst2src_size += pkt.payload_size
                
        flow.udps.lowaaf = lowaaf
        if flow.udps.src2dst_size != 0 and flow.udps.dst2src_size != 0:
            flow.udps.aaf = flow.udps.dst2src_size/flow.udps.src2dst_size

        if flow.udps.aaf != 0 and flow.udps.aaf < 0.1:
            lowaaf += 1
            


        if lowaaf > 100:
            flow.udps.expiration_id = -1 # -1 value force expiration
            flow.udps.alarm = True
            lowaaf = 0
            time = None

        if time is None:
            time = pkt.time
        time_diff = pkt.time - time

        if time_diff > 10000:
            lowaaf = 0
            time = None

In [5]:
class Frequency(NFPlugin):
    def on_init(self, pkt, flow):
        global lowflow
        global time
        global freq_alarm
        flow.expiration_id = 0
        flow.udps.fr_alarm = False
        if flow.bidirectional_duration_ms == 0:
            bidir = 0.0001
        else:
            bidir = flow.bidirectional_duration_ms
        frequency = flow.bidirectional_packets/bidir
        
        if frequency > 300:
            freq_alarm += 1
            time = None
        
    def on_update(self, pkt, flow):
        global lowflow
        global time
        global freq_alarm
        if flow.bidirectional_duration_ms == 0:
            bidir = 0.0001
        else:
            bidir = flow.bidirectional_duration_ms
        frequency = flow.bidirectional_packets/bidir
        
        if frequency > 300:
            freq_alarm += 1
            flow.udps.fr_alarm = True
            time = None

        if time is None:
            time = pkt.time
        time_diff = pkt.time - time

        if time_diff > 10000:
            freq_alarm = 0
            time = None
    

In [6]:
class LowDelta(NFPlugin):

    def on_init(self, pkt, flow):
        global count
        global time
        flow.expiration_id = 0
        flow.udps.exp = False
        if pkt.delta_time < 1:
            flow.udps.low_time_packets = 1
            count += 1
        else:
            flow.udps.low_time_packets = 0
        flow.udps.exp = False
        flow.udps.count = count

    def on_update(self, pkt, flow):
        global count
        global time
        if pkt.delta_time < 1:
            count += 1
            flow.udps.low_time_packets += 1

        flow.udps.exp = False
        flow.udps.count = count

        if count > 100:
            count = 0
            flow.udps.exp = True
            flow.expiration_id = -1 # -1 value force expiration
            
        if time is None:
            time = pkt.time
        time_diff = pkt.time - time

        if time_diff > 10000: #reset after 10s
            count = 0
            time = None


# Load Data

In [7]:
mssql_packetsize_pf = NFStreamer(source="/home/student/BA/PCAP-01-12/PCAP-01-12_0250-0499/SAT-01-12-2018_0444.pcap",
                         udps=PacketSizePerFlow(),
                         )
mssql_freq = NFStreamer(source="/home/student/BA/PCAP-01-12/PCAP-01-12_0250-0499/SAT-01-12-2018_0444.pcap",
                         udps=Frequency(),
                         )
mssql_lowdelta = NFStreamer(source="/home/student/BA/PCAP-01-12/PCAP-01-12_0250-0499/SAT-01-12-2018_0444.pcap",
                         udps=LowDelta(),
                         )
mssql_aaf = NFStreamer(source="/home/student/BA/PCAP-01-12/PCAP-01-12_0250-0499/SAT-01-12-2018_0444.pcap",
                         udps=AAF(),
                         )

#2013-12-17_capture1 
comparison_packetsize_pf = NFStreamer(source="/home/student/BA/Comparison-Traffic/2013-12-17_capture1.pcap",
                         udps=[PacketSizePerFlow()])
comparison_frequency = NFStreamer(source="/home/student/BA/Comparison-Traffic/2013-12-17_capture1.pcap",
                         udps=[Frequency()])

comparison_lowdelta = NFStreamer(source="/home/student/BA/Comparison-Traffic/2013-12-17_capture1.pcap",
                         udps=[LowDelta()])

comp_aaf = NFStreamer(source="/home/student/BA/Comparison-Traffic/2013-12-17_capture1.pcap",
                         udps=[AAF()])


# Testing

In [8]:
#testing aaf

a = 0


for flow in mssql_aaf:
   if flow.udps.alarm == True:
        a += 1

        
print("Number of Expirations mssql: ",a)


a = 0

for flow in comp_aaf:
   if flow.udps.alarm == True:
        a += 1

print("Number of Expirations comp: ",a)

Number of Expirations mssql:  0
Number of Expirations comp:  5


In [9]:
#testing packet_size
a = 0

for flow in mssql_packetsize_pf:
   if flow.udps.alarm == True:
        a += 1
        
print("Number of Expirations mssql: ",a)


a = 0

for flow in comparison_packetsize_pf:
   if flow.udps.alarm == True:
        a += 1
        
print("Number of Expirations comp: ",a)


Number of Expirations mssql:  1472
Number of Expirations comp:  0


In [10]:
#testing frequency

a  = 0

for flow in mssql_freq:
   if flow.udps.fr_alarm == True:
        a += 1

print("Number of Expirations mssql: ",a)

a  = 0

for flow in comparison_frequency:
   if flow.udps.fr_alarm == True:
        a += 1

print("Number of Expirations comp: ",a)


Number of Expirations mssql:  148202
Number of Expirations comp:  15


In [12]:
#testing low_delta

a  = 0

for flow in mssql_lowdelta:
   if flow.udps.exp == True:
        a += 1

print("Number of Expirations mssql: ",a)

a  = 0

for flow in comparison_lowdelta:
   if flow.udps.exp == True:
        a += 1
        
print("Number of Expirations comp: ",a)


Number of Expirations mssql:  2917
Number of Expirations comp:  2525
